In [ ]:
import numpy as np
import pandas as pd
import joblib

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
sample = (pd.read_csv("machine_learning/company_3/task_solution/derived/sample.csv")
          .drop(["id"], axis=1))

y_sample = sample["gb"]
X_sample = sample.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sample,
                                                    y_sample,
                                                    test_size=0.2,
                                                    random_state=1234,
                                                    stratify=y_sample)

In [ ]:
th_nans = 0.35
th_high_cardinality = 10
th_low_variance = 0.1
th_corr = 0.05

In [ ]:
cols_nans = (X_train
            .columns[X_train.isna().mean() > th_nans]
            .tolist())

In [ ]:
cats = X_train.columns[X_train.columns.str.startswith("cat")]

X_train.loc[:, cats] = X_train[cats].astype(str)
X_test.loc[:, cats] = X_test[cats].astype(str)

In [ ]:
cols_high_cardinality = (cats[X_train[cats]
                              .nunique() > th_high_cardinality]
                         .tolist())

In [ ]:
cols_to_drop_init = list(set(cols_nans) | set(cols_high_cardinality))

In [ ]:
X_train = X_train.drop(cols_to_drop_init, axis=1)
X_test = X_test.drop(cols_to_drop_init, axis=1)

In [ ]:
cats_left = X_train.select_dtypes(include="object").columns.tolist()
nums_left = X_train.select_dtypes(include=np.number).columns.tolist()

In [ ]:
X_train.loc[:, cats_left] = (X_train[cats_left]
                             .fillna(X_train[cats_left].mode()))

X_test.loc[:, cats_left] = (X_test[cats_left]
                             .fillna(X_train[cats_left].mode()))

In [ ]:
X_train.loc[:, nums_left] = (X_train[nums_left]
                             .fillna(X_train[nums_left].mean()))

X_test.loc[:, nums_left] = (X_test[nums_left]
                             .fillna(X_train[nums_left].mean()))

In [ ]:
print(X_train.isna().sum().sum())
print(X_test.isna().sum().sum())

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_train, X_test = X_train.align(X_test,
                                join="left",
                                axis=1,
                                fill_value=0)

In [ ]:
cols_low_std = (X_train
                .columns[(X_train.std() < th_low_variance) &
                         (X_train.columns != "gb")]
                .tolist())

In [ ]:
cols_low_corr = (X_train
                .columns[(X_train.corr().abs()["gb"] < th_corr) &
                         (X_train.columns != "gb")]
                .tolist())

In [ ]:
cols_to_drop_final = list(set(cols_low_std) | set(cols_low_corr))
cols_to_drop_final.append("gb")

In [ ]:
X_train = X_train.drop(cols_to_drop_final, axis=1)
X_test = X_test.drop(cols_to_drop_final, axis=1)

In [ ]:
target = y_train.value_counts()
spw = target[0] / target[1]

xgb_model = XGBClassifier(random_state=1234,
                          objective="binary:logistic",
                          scale_pos_weight=spw,
                          n_jobs=-1)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print("ROC-AUC on test: {}".format(roc_auc_score(y_test, y_pred)))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
joblib.dump(xgb_model, "machine_learning/company_3/task_solution/derived/base_model.sav")
